<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

# Setup


In [1]:
credentials = {
    "bucket": "BUCKET",
    "access_key_id": "ACCESS_KEY_ID",
    "secret_access_key": "SECRET_ACCESS_KEY",
    "endpoint_url": "ENDPOINT_URL"
}

In [2]:
NUM_TRAIN_STEPS = 500

import os
CLOUD_ANNOTATIONS_MOUNT = os.path.join('content', credentials['bucket'])
ANNOTATIONS_JSON_PATH   = os.path.join(CLOUD_ANNOTATIONS_MOUNT, '_annotations.json')

DATA_PATH         = 'content/data'
LABEL_MAP_PATH    = os.path.join(DATA_PATH, 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join(DATA_PATH, 'train.record')
VAL_RECORD_PATH   = os.path.join(DATA_PATH, 'val.record')

# Install the TensorFlow Object Detection API
In order to use the TensorFlow Object Detection API, we need to clone it's GitHub Repo.

### Dependencies
Most of the dependencies required come preloaded in Google Colab. The only additional package we need to install is TensorFlow.js, which is used for converting our trained model to a model that is compatible for the web.

### Protocol Buffers
The TensorFlow Object Detection API relies on what are called `protocol buffers` (also known as `protobufs`). Protobufs are a language neutral way to describe information. That means you can write a protobuf once and then compile it to be used with other languages, like Python, Java or C.

The `protoc` command used below is compiling all the protocol buffers in the `object_detection/protos` folder for Python.

### Environment
To use the object detection api we need to add it to our `PYTHONPATH` along with `slim` which contains code for training and evaluating several widely used Convolutional Neural Network (CNN) image classification models.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
import os
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/tensorflow/models

!pip install cloud-annotations==0.0.4
!pip install tf_slim
!pip install --no-deps tensorflowjs==1.4.0

%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'

Cloning into 'models'...


remote: Enumerating objects: 2280, done.


remote: Counting objects: 100% (2280/2280), done.


remote: Compressing objects: 100% (1971/1971), done.


remote: Total 2280 (delta 557), reused 969 (delta 283), pack-reused 0
Receiving objects: 100% (2280/2280), 30.55 MiB | 19.72 MiB/s, done.
Resolving deltas: 100% (557/557), done.


...

/bin/sh: pip: command not found


/bin/sh: pip: command not found


[Errno 2] No such file or directory: '/content/models/research'
/Users/niko/Documents/cloud-annotations-elyra


object_detection/protos/*.proto: No such file or directory


KeyError: 'PYTHONPATH'

# Mount Cloud Annotations Bucket
In order to use files from Cloud Annotations we need to mount it to Colab.

In [ ]:
import cloud_annotations as ca
ca.mount(CLOUD_ANNOTATIONS_MOUNT, credentials)

# Generate a Label Map
One piece of data the Object Detection API needs is a label map protobuf. The label map associates an integer id to the text representation of the label. The ids are indexed by 1, meaning the first label will have an id of 1 not 0.

Here is an example of what a label map looks like:
```
item {
  id: 1
  name: 'Cat'
}

item {
  id: 2
  name: 'Dog'
}

item {
  id: 3
  name: 'Gold Fish'
}
```


In [ ]:
import os
import json

# Get a list of labels from the annotations.json
labels = {}
with open(ANNOTATIONS_JSON_PATH) as f:
    annotations = json.load(f)
    labels = annotations['labels']

# Create a file named label_map.pbtxt
os.makedirs(DATA_PATH, exist_ok=True)
with open(LABEL_MAP_PATH, 'w') as f:
    # Loop through all of the labels and write each label to the file with an id
    for idx, label in enumerate(labels):
        f.write('item {\n')
        f.write("\tname: '{}'\n".format(label))
        f.write('\tid: {}\n'.format(idx + 1)) # indexes must start at 1
        f.write('}\n')

# Generate TFRecords
The TensorFlow Object Detection API expects our data to be in the format of TFRecords.

The TFRecord format is a collection of serialized feature dicts, one for each image, looking something like this:
```
{
  'image/height': 1800,
  'image/width': 2400,
  'image/filename': 'image1.jpg',
  'image/source_id': 'image1.jpg',
  'image/encoded': ACTUAL_ENCODED_IMAGE_DATA_AS_BYTES,
  'image/format': 'jpeg',
  'image/object/bbox/xmin': [0.7255949630314233, 0.8845598428835489],
  'image/object/bbox/xmax': [0.9695875693160814, 1.0000000000000000],
  'image/object/bbox/ymin': [0.5820120073891626, 0.1829972290640394],
  'image/object/bbox/ymax': [1.0000000000000000, 0.9662484605911330],
  'image/object/class/text': (['Cat', 'Dog']),
  'image/object/class/label': ([1, 2])
}
```

In [ ]:
import os
import io
import json
import random

import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def create_tf_record(images, annotations, label_map, image_path, output):
    # Create a train.record TFRecord file.
    with tf.python_io.TFRecordWriter(output) as writer:
        # Loop through all the training examples.
        for image_name in images:
            try:
                # Make sure the image is actually a file
                img_path = os.path.join(image_path, image_name)   
                if not os.path.isfile(img_path):
                    continue
          
                # Read in the image.
                with tf.gfile.GFile(img_path, 'rb') as fid:
                    encoded_jpg = fid.read()

                # Open the image with PIL so we can check that it's a jpeg and get the image
                # dimensions.
                encoded_jpg_io = io.BytesIO(encoded_jpg)
                image = PIL.Image.open(encoded_jpg_io)
                if image.format != 'JPEG':
                    raise ValueError('Image format not JPEG')

                width, height = image.size

                # Initialize all the arrays.
                xmins = []
                xmaxs = []
                ymins = []
                ymaxs = []
                classes_text = []
                classes = []

                # The class text is the label name and the class is the id. If there are 3
                # cats in the image and 1 dog, it may look something like this:
                # classes_text = ['Cat', 'Cat', 'Dog', 'Cat']
                # classes      = [  1  ,   1  ,   2  ,   1  ]

                # For each image, loop through all the annotations and append their values.
                for a in annotations[image_name]:
                    if ("x" in a and "x2" in a and "y" in a and "y2" in a):
                        label = a['label']
                        xmins.append(a["x"])
                        xmaxs.append(a["x2"])
                        ymins.append(a["y"])
                        ymaxs.append(a["y2"])
                        classes_text.append(label.encode("utf8"))
                        classes.append(label_map[label])
       
                # Create the TFExample.
                tf_example = tf.train.Example(features=tf.train.Features(feature={
                    'image/height': dataset_util.int64_feature(height),
                    'image/width': dataset_util.int64_feature(width),
                    'image/filename': dataset_util.bytes_feature(image_name.encode('utf8')),
                    'image/source_id': dataset_util.bytes_feature(image_name.encode('utf8')),
                    'image/encoded': dataset_util.bytes_feature(encoded_jpg),
                    'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
                    'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
                    'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
                    'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
                    'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
                    'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
                    'image/object/class/label': dataset_util.int64_list_feature(classes),
                }))
                if tf_example:
                    # Write the TFExample to the TFRecord.
                    writer.write(tf_example.SerializeToString())
            except ValueError:
                print('Invalid example, ignoring.')
                pass
            except IOError:
                print("Can't read example, ignoring.")
                pass

with open(ANNOTATIONS_JSON_PATH) as f:
    annotations = json.load(f)['annotations']
    image_files = [image for image in annotations.keys()]
    # Load the label map we created.
    label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

    random.seed(42)
    random.shuffle(image_files)
    num_train = int(0.7 * len(image_files))
    train_examples = image_files[:num_train]
    val_examples = image_files[num_train:]

    create_tf_record(train_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, TRAIN_RECORD_PATH)
    create_tf_record(val_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, VAL_RECORD_PATH)

# Model config
The final thing we need to do is inject our pipline with the amount of labels we have and where to find the label map, TFRecord and model checkpoint. We also need to change the the batch size, because the default batch size of 128 is too large for Colab to handle.

In [ ]:
import re

from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = '/content/models/research/object_detection/samples/configs/'
pipeline_skeleton = pipeline_skeleton + 'ssd_mobilenet_v1_quantized_300x300_coco14_sync.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
    'model.{}.num_classes'.format(meta_arch): num_classes,
    'train_config.batch_size': 24,
    'train_input_path': TRAIN_RECORD_PATH,
    'eval_input_path': VAL_RECORD_PATH,
    'train_config.fine_tune_checkpoint': 'content/checkpoint/model.ckpt',
    'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, DATA_PATH)